In [41]:
import asyncio
import websockets
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import nest_asyncio
nest_asyncio.apply()
# 创建一个空的 DataFrame 用于存储 K 线数据
df = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close'])

# 初始化一个 Plotly 蜡烛图对象
fig = go.Figure()

# 初始化图表布局并绘制初始的空蜡烛图
def initialize_chart():
    fig.update_layout(
        title='BTC/USDT 实时 K 线图',
        xaxis_title='时间',
        yaxis_title='价格 (USDT)',
        xaxis_rangeslider_visible=False
    )
    # 绘制空图以初始化
    fig.add_trace(go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='BTC/USDT'
    ))
    # 显示初始的空图
    pio.show(fig)

# 实时更新图表
def update_chart():
    fig.data[0].x = df['timestamp']
    fig.data[0].open = df['open']
    fig.data[0].high = df['high']
    fig.data[0].low = df['low']
    fig.data[0].close = df['close']
    fig.show()

# WebSocket 处理函数，接收实时数据并更新 DataFrame 和图表
async def binance_kline():
    uri = "wss://stream.binance.com:9443/ws/btcusdt@kline_1m"  # Binance 1分钟K线
    async with websockets.connect(uri) as websocket:
        global df
        while True:
            # 接收WebSocket数据
            message = await websocket.recv()
            data = json.loads(message)
            
            # 解析K线数据
            kline = data['k']
            is_kline_closed = kline['x']  # K线是否结束
            timestamp = pd.to_datetime(kline['t'], unit='ms')
            open_price = float(kline['o'])
            high_price = float(kline['h'])
            low_price = float(kline['l'])
            close_price = float(kline['c'])

            # 构建新的一行数据
            new_row = {
                'timestamp': timestamp,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price
            }

            # 如果K线结束，更新DataFrame并刷新图表
            if is_kline_closed:
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                update_chart()  # 动态刷新蜡烛图

# 初始化图表
initialize_chart()

# 运行 WebSocket，接收并处理数据
asyncio.run(binance_kline())


/tmp/ipykernel_149383/2236878824.py:74: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



KeyboardInterrupt: 